# Threads and Processing

**Python uses the threading module for threading.** 

**Keep in mind that due to the Global Interpreter Lock (GIL), Python threads are best for IO-bound tasks, not CPU-bound ones.**


Use the multiprocessing module for creating separate **processes**. This avoids the GIL and is better for CPU-bound tasks.

### What is the Global Interpreter Lock (GIL) in Python?

The Global Interpreter Lock (GIL) is a mechanism used in the <br>
Python (the default and most widely used version). It allows only one thread <br>
to execute Python bytecode at a time, even on multi-core processors.

**Why Does the GIL Exist?**

The GIL simplifies memory management in CPython. Specifically:

    Python uses reference counting for garbage collection.

    Managing shared memory (especially reference counts) across threads without a lock would be error-prone and complex.

So, to avoid race conditions in memory management, the GIL ensures that only one thread touches Python objects at a time.

### What is Reference Counting in Python?

In Python (especially CPython, the standard implementation), reference counting is the primary mechanism used for memory management and garbage collection.

🔗 A reference count is:

    The number of references (or "pointers") to an object in memory.

When an object is created, Python keeps track of how many things (variables, containers, etc.) are pointing to it.

🔄 How Reference Counting Works

Example:





In [1]:
a = [1, 2, 3]  # A list is created, ref count = 1

b = a          # Now both `a` and `b` point to the same list, ref count = 2

del a          # `a` is deleted, ref count = 1

del b          # `b` is deleted, ref count = 0 → object is destroyed

When the reference count drops to zero, Python automatically frees the memory used by that object.



### So now we will understand threads are how used in I/O bound  tasks

An I/O-bound task is a program or operation whose performance is limited by input/output (I/O) operations, rather than the CPU's speed.

🔁 I/O means:

    Reading/writing files

    Network communication (e.g., downloading a web page)

    Database access

    User input

    Disk or USB operations
    

🐢 I/O is slow compared to the CPU

So even if your CPU is fast, your program is waiting around for slow things like:

    A file to load

    A web request to finish

    A database to respond



### 🧵 Why Threads Help with I/O-Bound Tasks

In Python, threads can run while another thread waits for I/O to finish.
Example:

If one thread is waiting for a file to be read, another thread can keep running.<br>

This makes your program more efficient and responsive, especially in web servers or GUIs.

In [3]:
import time
import threading

def fetch_data():
    print("Start fetching...")
    time.sleep(3)  # Simulate I/O delay
    print("Done fetching!")

start = time.time()

# Run two I/O-bound tasks in threads
t1 = threading.Thread(target=fetch_data)
t2 = threading.Thread(target=fetch_data)

t1.start()
t2.start()
t1.join()
t2.join()

print(f"Total time: {time.time() - start:.2f} seconds")


Start fetching...
Start fetching...
Done fetching!
Done fetching!
Total time: 3.01 seconds


Without threads, this would take ~6 seconds. With threads, they run concurrently, finishing in ~3 seconds.

### What the Code Does:
1. Import Modules



In [4]:
import time
import threading


time is used to simulate delay and measure how long the code takes.

threading lets us run tasks in separate threads.

#### Define a Simulated I/O Task

In [5]:
def fetch_data():
    print("Start fetching...")
    time.sleep(3)  # Waits for 3 seconds (simulating I/O)
    print("Done fetching!")


This function simulates something like downloading a file or querying a database.

time.sleep(3) represents waiting (which is typical in I/O tasks).

#### Record Start Time

In [6]:
start = time.time()


Saves the current time so we can measure total runtime later.

#### Create Threads

In [8]:
t1 = threading.Thread(target=fetch_data)
t2 = threading.Thread(target=fetch_data)


We create two threads that will both run the fetch_data function independently.

#### Start Threads

In [9]:
t1.start()
t2.start()


Start fetching...
Start fetching...
Done fetching!
Done fetching!


Both threads begin execution.

They both print “Start fetching...” and then sleep for 3 seconds.

These delays overlap, because threads run concurrently.

In [10]:
t1.join()
t2.join()


.join() ensures the main program waits until both threads are done before continuing.

#### Measure Total Time

In [ ]:
print(f"Total time: {time.time() - start:.2f} seconds")


Prints the time it took to run everything.

This example shows how multi-threading helps speed up I/O-bound tasks by overlapping the waiting times.

#### Version WITHOUT Threads (Sequential Execution)

In [12]:
import time

def fetch_data():
    print("Start fetching...")
    time.sleep(3)  # Simulate I/O delay
    print("Done fetching!")

start = time.time()

# Run two fetch operations one after the other
fetch_data()
fetch_data()

print(f"Total time: {time.time() - start:.2f} seconds")


Start fetching...
Done fetching!
Start fetching...
Done fetching!
Total time: 6.01 seconds


🧩 What’s Different?

    We call fetch_data() twice directly, one after the other.

    Each call to fetch_data() takes 3 seconds.

    Because they run sequentially, the total time is 3 + 3 = 6 seconds.



### Key Differences in Python
|Feature	|threading	|multiprocessing|
|----------|-----------|----------------|
|Best for	|I/O-bound tasks	|CPU-bound tasks|
|Memory	|Shared within a process	|Separate memory for each process|
|Performance	|Limited by GIL for CPU tasks	|Bypasses GIL, better for CPU tasks|
|Crash Risk	|One bad thread can crash others	|Crashes are isolated between processes|